# BERTopic model

In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN

import tensorflow as tf
# from cuml.cluster import HDBSCAN
# from cuml.manifold import UMAP
# from cuml.preprocessing import normalize

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer 
import string
import re
import nltk
from bs4 import BeautifulSoup   
import contractions,unicodedata


nltk.download('stopwords')                              # Download Stopwords.
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import PorterStemmer                     # Stemmer
from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.


import spacy
from sklearn.cluster import KMeans
import plotly.express as px

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data= pd.read_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\mask_name.csv')

In [3]:
data.head()

,Unnamed: 0,email_id,email,subject
0,0,allen-p\all_documents\100#2,"PERSON , \r\n\r\n Please use the second check ...",Consolidated positions : Issues & To Do list
1,1,allen-p\all_documents\101#2,STRUCTURE : \r\n Typically the structure is a ...,Consolidated positions : Issues & To Do list
2,3,allen-p\all_documents\103#1,---------------------- Forwarded by PERSON on ...,Re : 2001 Margin Plan
3,4,allen-p\all_documents\104#1,Internet Data Gain Is a Major Power Drain on \...,"Var, Reporting and Resources Meeting"
4,5,allen-p\all_documents\105#1,Do not underestimate the effects of the Intern...,Westgate


## Data processing

In [4]:
data['email'] = data['email'].str.strip()
#data['subject'] = data['subject'].str.strip()

In [5]:
# utl removal
def remove_url(text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    #text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(regex, '', text, flags=re.MULTILINE)
    return text
data['clean_text'] = data['email'].apply(lambda x: remove_url(x))

In [6]:
# html tag removal
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

data['clean_text'] = data['clean_text'].apply(lambda x: strip_html(x))

In [7]:
# contractions removal
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

data['clean_text'] = data['clean_text'].apply(lambda x: replace_contractions(x))
data.head()

,Unnamed: 0,email_id,email,subject,clean_text
0,0,allen-p\all_documents\100#2,"PERSON , \r\n\r\n Please use the second check ...",Consolidated positions : Issues & To Do list,"PERSON , \r\n\r\n Please use the second check ..."
1,1,allen-p\all_documents\101#2,STRUCTURE : \r\n Typically the structure is a ...,Consolidated positions : Issues & To Do list,STRUCTURE : \r\n Typically the structure is a ...
2,3,allen-p\all_documents\103#1,---------------------- Forwarded by PERSON on ...,Re : 2001 Margin Plan,---------------------- Forwarded by PERSON on ...
3,4,allen-p\all_documents\104#1,Internet Data Gain Is a Major Power Drain on \...,"Var, Reporting and Resources Meeting",Internet Data Gain Is a Major Power Drain on \...
4,5,allen-p\all_documents\105#1,Do not underestimate the effects of the Intern...,Westgate,Do not underestimate the effects of the Intern...


In [8]:
# Removing numbers
def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text

data['clean_text'] = data['clean_text'].apply(lambda x: remove_numbers(x))

In [9]:
# tokenization
data['clean_text'] = data.apply(lambda row: nltk.word_tokenize(row['clean_text']), axis=1) 

In [10]:
%%time
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

def lemmatize_list(words):
    new_words = []
    for word in words:
        new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

# def mask_names(words):
#     new_words = []
#     for word in words:
#         doc=nlp(word)
#         if doc[0].ent_type_=='PERSON':
#             new_words.append('PERSON')
#         else:
#             new_words.append(word)
#     return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize_list(words)
#     words = mask_names(words)
    return ' '.join(words)

data['clean_text'] = data.apply(lambda row: normalize(row['clean_text']), axis=1)
data.head()

CPU times: total: 19.4 s
Wall time: 32.9 s


,Unnamed: 0,email_id,email,subject,clean_text
0,0,allen-p\all_documents\100#2,"PERSON , \r\n\r\n Please use the second check ...",Consolidated positions : Issues & To Do list,person please use second check october payment...
1,1,allen-p\all_documents\101#2,STRUCTURE : \r\n Typically the structure is a ...,Consolidated positions : Issues & To Do list,structure typically structure limit partnershi...
2,3,allen-p\all_documents\103#1,---------------------- Forwarded by PERSON on ...,Re : 2001 Margin Plan,forward person pm invitation chairperson perso...
3,4,allen-p\all_documents\104#1,Internet Data Gain Is a Major Power Drain on \...,"Var, Reporting and Resources Meeting",internet data gain major power drain local uti...
4,5,allen-p\all_documents\105#1,Do not underestimate the effects of the Intern...,Westgate,underestimate effect internet economy load gro...


In [213]:
# drop NAs
data.dropna(subset=['clean_text'], inplace=True)

In [11]:
nlp = spacy.load('en_core_web_trf')
nlp.add_pipe("merge_entities")

def get_ner(text):
    doc = nlp(text)
    return " ".join(['PERSON' if t.ent_type_ and t.ent_type_ == 'PERSON' else t.text for t in doc])


In [214]:
#data.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\mask_name_cleaned.csv',index_label=False)

In [2]:
data= pd.read_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\mask_name_cleaned.csv')
data.head()

,Unnamed: 0,email_id,email,subject,clean_text
0,0,allen-p\all_documents\100#2,"PERSON , \r\n\r\n Please use the second check ...",Consolidated positions : Issues & To Do list,person please use second check october payment...
1,1,allen-p\all_documents\101#2,STRUCTURE : \r\n Typically the structure is a ...,Consolidated positions : Issues & To Do list,structure typically structure limit partnershi...
2,3,allen-p\all_documents\103#1,---------------------- Forwarded by PERSON on ...,Re : 2001 Margin Plan,forward person pm invitation chairperson perso...
3,4,allen-p\all_documents\104#1,Internet Data Gain Is a Major Power Drain on \...,"Var, Reporting and Resources Meeting",internet data gain major power drain local uti...
4,5,allen-p\all_documents\105#1,Do not underestimate the effects of the Intern...,Westgate,underestimate effect internet economy load gro...


In [3]:
#data2= data.sample(n=20000)
if type(data['clean_text']) is list:
    text = data['clean_text']
else:
    text = data['clean_text'].tolist()

text=[str(x) for x in text]

## Model

In [4]:
# Defining the grid to implement Random Search CV

## BERTopic hyperparameters
# transformers. Taken from https://www.sbert.net/docs/pretrained_models.html
s_transf=['all-MiniLM-L6-v2','all-MiniLM-L12-v2']

# top n words per topic, it is recommended to keep this number between 10-20
top_n_words=[int(x) for x in np.linspace(10,20,6)]

# ngram range: Let's try 1,2 and 3-grams
ngram_r=[(1,1),(1,2),(1,3)]

# min topic size
min_topic_s=[int(x) for x in np.linspace(10,200,6)]

## UMAP hyperparameters
# number of neighbors
n_neigh=[int(x) for x in np.linspace(10,50,6)]

## HDBSCAN hyperparameters
# min cluster size
min_cl_s=[int(x) for x in np.linspace(50,250,6)]



In [61]:
# %%time
# with tf.device('/GPU:0'):
#     # embedding
#     #sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
#     sentence_model = SentenceTransformer('all-MiniLM-L12-v2')
#     embeddings = sentence_model.encode(text, show_progress_bar=True)

# all_embeddings = np.array(embeddings)
# np.save('embeddings_L12.npy', all_embeddings)
#embeddings= np.load('embeddings.npy')
embeddings= np.load('embeddings_L12.npy')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/573 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1025 [00:00<?, ?it/s]

CPU times: total: 51min 43s
Wall time: 7min 52s


In [63]:
def create_model(topwords,n_gram,min_top, nn, min_cluster):
 
    # ngrams
    vectorizer_model = CountVectorizer(ngram_range=n_gram, stop_words="english")
    
    # UMAP: dimensionality reduction
    umap_model = UMAP(n_neighbors=nn, min_dist=0.0, metric='cosine', random_state=42)
    
    # HDBSCAN: clustering
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster, metric='euclidean', 
                            cluster_selection_method='eom', prediction_data=True, min_samples=5)
    
    cluster_model = KMeans(n_clusters=min_top)
    
    # BERTopic
    model = BERTopic(
        top_n_words=topwords,
        #min_topic_size=min_top,
        nr_topics=min_top,
        vectorizer_model=vectorizer_model,
        language='english', calculate_probabilities=True,
        #embedding_model=sentence_model,
        verbose=True,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
    )
    
    return model

In [104]:
model=create_model(20,ngram_r[2],min_topic_s[0],10,min_cl_s[0])
model

In [105]:
%%time
with tf.device('/GPU:0'):
    topics, probs = model.fit_transform(text,embeddings)


2022-10-24 18:50:51,571 - BERTopic - Reduced dimensionality
2022-10-24 18:51:22,194 - BERTopic - Clustered reduced embeddings
2022-10-24 18:52:03,063 - BERTopic - Reduced number of topics from 138 to 11


CPU times: total: 1min 46s
Wall time: 1min 28s


In [106]:
# save the model
aux=[s_transf[1],20,ngram_r[2],min_topic_s[0],10,min_cl_s[0]]
aux=[str(x) for x in aux]
model.save("_".join(aux))
#model=BERTopic.load("_".join(aux))


C:\Users\p_uli\anaconda3\envs\tf-gpu\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [107]:
# probs=model.probabilities_
# topics=model.topics_

## Results

In [108]:
freq = model.get_topic_info()
freq.head(11)

,Topic,Count,Name
0,-1,20143,-1_person_enron_company_new
1,0,3741,0_person_nt_know_think
2,1,1923,1_intend_recipient_person_intend recipient
3,2,1407,2_meet_person_pm_person person
4,3,1078,3_energy_person_power_california
5,4,982,4_enron_person_enron north america_enron north
6,5,965,5_fyi_send_person_fyi fyi
7,6,783,6_deal_person_price_volume
8,7,676,7_person_agreement_credit_contract
9,8,588,8_thank_person_thank person_person thank


In [109]:
# create topic assignment per email
topics=[]
probas=[]
for i in range(data.shape[0]):
    topics.append(np.argmax(probs[i]))
    probas.append(np.max(probs[i]))


In [110]:
# create the table with the 20 most important words per topic
words=[]
c_tfidf=[]
top=[]
for i in range(0,10):
    top= top+ [i]*20
    for x in model.get_topic(i): #c-TF-IDF scores
        words.append(x[0])
        c_tfidf.append(x[1])
    


In [111]:
# 
data['topic']=topics
data['probs']= probas

In [112]:
# topic words
top_words_topic=pd.DataFrame(list(zip(top,words,c_tfidf)),columns=['topic','top_words','c-tf-idf'])

In [113]:
#file_name=[s_transf[0],top_n_words[0],ngram_r[0],min_topic_s[0],10,min_cl_s[0]]
file_name=[s_transf[1],20,ngram_r[2],min_topic_s[0],10,min_cl_s[0]]
file_name=[str(x) for x in file_name]
file_name="_".join(file_name)


In [114]:
top_words_topic.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\BERTopic\top_words_{}.csv'.format(file_name),index=False)
data.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\BERTopic\assignments_{}.csv'.format(file_name),index=False)

## Visualization

In [115]:
fig=model.visualize_topics()
fig.write_html(r"C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Images\BERTopic\Distance_{}.html".format(file_name))

In [116]:
hierarchy=model.visualize_hierarchy()
hierarchy.write_html(r"C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Images\BERTopic\Hierarchy_{}.html".format(file_name))

In [117]:
docs_viz=model.visualize_documents(text,embeddings=embeddings)
docs_viz.write_html(r"C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Images\BERTopic\Docs_{}.html".format(file_name))

In [118]:
bars=model.visualize_barchart(top_n_topics=10)
bars.write_html(r"C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Images\BERTopic\Bars_{}.html".format(file_name))


In [147]:
# # Document visualization (left fot the best models)

# from umap import UMAP
# # Prepare embeddings

# sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# embeddings = sentence_model.encode(text, show_progress_bar=False)

# # Train BERTopic
# topic_model = BERTopic(nr_topics=10,).fit(text, embeddings)

# # Run the visualization with the original embeddings
# topic_model.visualize_documents(text, embeddings=embeddings)

# # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(text, reduced_embeddings=reduced_embeddings)
